In [ ]:
# pip install requests bs4 lxml xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from requests.compat import urljoin
from bs4 import BeautifulSoup
import re
import xlsxwriter
import time
import csv

In [3]:
def CleanData(InfoString):
    InfoString = InfoString.replace('\n', '',)  
    InfoString = InfoString.replace('\r', '',)  
    InfoString = InfoString.replace(' ', '',)
    InfoString = InfoString.replace('年', '-', 1)  
    InfoString = InfoString.replace('月', '-', 1)
    InfoString = InfoString.replace('日', '', 1)
    return InfoString

def ExtractBJWeather():
    # Scrape Beijing weather data
    output_file = 'Beijing_Weather_Data.csv'
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', 'Weather Condition', 'Temperature', 'Wind Force and Direction'])  # CSV headers

        url = 'http://www.tianqihoubao.com/lishi/beijing.html'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
            'Connection': 'close'
        }
        html = requests.get(url, headers=headers)
        html.encoding = 'gb2312'  # Correctly set the encoding to match the website
        bsObj = BeautifulSoup(html.text, 'lxml')  # Use text instead of content with the correct encoding

        year_weather_data = bsObj.find_all('div', class_="box pcity")  # Weather data for all years
        year = 2010
        for quarter_data in year_weather_data:  # Iterate through quarterly data
            year += 1
            if year <= 2011:  # Restrict to data up to 2020
                quarter_list = quarter_data.find_all('ul')  # Quarterly data
                for month_data in quarter_list:  # Iterate through months in a quarter
                    month_links = month_data.find_all('a')  # Extract links for each month
                    for link in month_links:
                        month_url = urljoin('http://www.tianqihoubao.com', link['href'])
                        try:
                            month_html = requests.get(month_url, headers=headers)
                            month_html.encoding = 'gb2312'  # Correct encoding
                            month_bsObj = BeautifulSoup(month_html.text, 'lxml')
                            month_rows = month_bsObj.find_all('tr')  # Rows of monthly data
                            for i, day_row in enumerate(month_rows):
                                if i == 0:  # Skip the header row
                                    continue
                                day_data = [CleanData(td.get_text()) for td in day_row.find_all('td')]
                                if len(day_data) == 4:
                                    writer.writerow(day_data)  # Write data to CSV file
                        except Exception as e:  # Handle request rate limit
                            print("Request rate too high, sleeping for 5 seconds...")
                            time.sleep(5)
                            print("Retrying...")
                            continue
                print(f'Finished scraping data for {year}...')
            else:
                break
    print('---------------------Scraping Complete----------------')
    return


ExtractBJWeather()   

Finished scraping data for 2011...
---------------------Scraping Complete----------------


In [5]:
import csv
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin

def CleanData(InfoString):
    """Clean the data by removing unwanted characters"""
    InfoString = InfoString.replace('\n', '',)  
    InfoString = InfoString.replace('\r', '',)  
    InfoString = InfoString.replace(' ', '',)
    InfoString = InfoString.replace('年', '-', 1)  
    InfoString = InfoString.replace('月', '-', 1)
    InfoString = InfoString.replace('日', '', 1)
    return InfoString

def ExtractWeatherForCity(city):
    """Scrape weather data for the specified city and return a list of records"""
    url = f'http://www.tianqihoubao.com/lishi/{city}.html'
    
    data_records = []  # List to hold the weather data

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
        'Connection': 'close'
    }
    html = requests.get(url, headers=headers)
    html.encoding = 'gb2312'  # Correctly set the encoding to match the website
    bsObj = BeautifulSoup(html.text, 'lxml')

    year_weather_data = bsObj.find_all('div', class_="box pcity")  # Weather data for all years
    year = 2010
    for quarter_data in year_weather_data:  # Iterate through quarterly data
        year += 1
        if year <= 2011:  # Restrict to data up to 2020
            quarter_list = quarter_data.find_all('ul')  # Quarterly data
            for month_data in quarter_list:  # Iterate through months in a quarter
                month_links = month_data.find_all('a')  # Extract links for each month
                for link in month_links:
                    month_url = urljoin('http://www.tianqihoubao.com', link['href'])
                    try:
                        month_html = requests.get(month_url, headers=headers)
                        month_html.encoding = 'gb2312'  # Correct encoding
                        month_bsObj = BeautifulSoup(month_html.text, 'lxml')
                        month_rows = month_bsObj.find_all('tr')  # Rows of monthly data
                        for i, day_row in enumerate(month_rows):
                            if i == 0:  # Skip the header row
                                continue
                            day_data = [CleanData(td.get_text()) for td in day_row.find_all('td')]
                            if len(day_data) == 4:
                                day_data.insert(1, city)  # Insert city name at the second column
                                data_records.append(day_data)  # Add to the list of records
                    except Exception as e:  # Handle request rate limit
                        print(f"Error scraping {city} data for month: {link['href']}, retrying...")
                        time.sleep(5)
                        continue
            print(f'Finished scraping data for {city}:{year}...')
        else:
            break

    return data_records

def WriteWeatherDataToCSV(city_list):
    """Scrape weather data for all cities and write them to a CSV file"""
    all_data = []  # List to hold data from all cities

    # Loop over each city in the city_list and scrape its weather data
    for city in city_list:
        city_data = ExtractWeatherForCity(city)
        all_data.extend(city_data)  # Append the data of this city to the overall list

    # Write all collected data to a CSV file at once
    output_file = 'Weather_Data_All_Cities.csv'
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', 'City', 'Weather Condition', 'Temperature', 'Wind Force and Direction'])  # CSV headers
        writer.writerows(all_data)  # Write all records at once

    print(f'Weather data for all cities has been written to {output_file}')

city_list = ['beijing', 'shanghai', 'guangzhou']
WriteWeatherDataToCSV(city_list)

Finished scraping data for beijing:2011...
Finished scraping data for shanghai:2011...
Finished scraping data for guangzhou:2011...
Weather data for all cities has been written to Weather_Data_All_Cities.csv
